In [2]:
import os

# Folder output utama
code_folders = ["speck128rfida", "speck128rfidb", "speck128rfidc", "speck128rfidd", "speck128rfide"]

# Buat folder utama jika belum ada
for folder in code_folders:
    os.makedirs(folder, exist_ok=True)

def generate_code_with_decrementing_plaintext(file_path, id_values):
    # Template kode Arduino
    code_template = f"""
#include <SPI.h>
#include <MFRC522.h>
#include <Wire.h>
#include <Adafruit_INA219.h>
#include <Crypto.h>
#include <Speck.h>
#include <string.h>
#include <MemoryFree.h>
#include <LiquidCrystal_I2C.h>

#define SS_PIN 10
#define RST_PIN 9

MFRC522 rfid(SS_PIN, RST_PIN);
Adafruit_INA219 ina219;
LiquidCrystal_I2C lcd(0x27, 16, 2);

Speck speck128;
byte buffer[16];

byte ciphertext[{id_values}];
byte decryptedtext[{id_values}];
unsigned long encryptionTime;
unsigned long decryptionTime;

float encryptionVoltage;
float encryptionCurrent;
float decryptionVoltage;
float decryptionCurrent;

const byte keySpeck128[16] = {{ 0x0f, 0x0e, 0x0d, 0x0c, 0x0b, 0x0a, 0x09, 0x08,
                              0x07, 0x06, 0x05, 0x04, 0x03, 0x02, 0x01, 0x00 }};

void encryptAndPrint(BlockCipher *cipher, const byte *key, size_t keySize, byte *plaintext, size_t plaintextSize) {{
  unsigned long startTime = micros();
  cipher->setKey(key, keySize);

  for (size_t i = 0; i < plaintextSize; i += 16) {{
    cipher->encryptBlock(buffer, &plaintext[i]);
    memcpy(&ciphertext[i], buffer, 16);
  }}

  unsigned long endTime = micros();
  encryptionVoltage = ina219.getBusVoltage_V();
  encryptionCurrent = ina219.getCurrent_mA();
  encryptionTime = endTime - startTime;

  // Output encrypted data (ciphertext)
  Serial.print(F("Ciphertext: "));
  for (size_t i = 0; i < plaintextSize; i++) {{
    if (ciphertext[i] < 0x10) Serial.print("0");
    Serial.print(ciphertext[i], HEX);
    Serial.print(" ");
  }}
  Serial.println();
  Serial.print(F("Encryption Completed in "));
  Serial.print(encryptionTime);
  Serial.println(F(" us"));
}}

void decryptAndPrint(BlockCipher *cipher, const byte *key, size_t keySize, byte *ciphertext, size_t ciphertextSize) {{
    unsigned long startdecTime = micros();
    cipher->setKey(key, keySize);

    for (size_t i = 0; i < ciphertextSize; i += 16) {{
        cipher->decryptBlock(buffer, &ciphertext[i]);
        memcpy(&decryptedtext[i], buffer, 16);
    }}

    unsigned long enddecTime = micros();
    decryptionVoltage = ina219.getBusVoltage_V();
    decryptionCurrent = ina219.getCurrent_mA();
    decryptionTime = enddecTime - startdecTime;

    // Output decrypted data
    Serial.print(F("Decrypted Text: "));
    for (size_t i = 0; i < ciphertextSize; i++) {{
        if (decryptedtext[i] < 0x10) Serial.print("0");
        Serial.print(decryptedtext[i], HEX);
        Serial.print(" ");
    }}
    Serial.println();
    Serial.print(F("Decryption Completed in "));
    Serial.print(decryptionTime);
    Serial.println(F(" us"));
}}

void displayData() {{
    float encryptionPower = encryptionVoltage * (encryptionCurrent / 1000);
    float decryptionPower = decryptionVoltage * (decryptionCurrent / 1000);
    int memoryUsage = 2048 - freeMemory();

    Serial.print(F("Memory Usage = "));
    Serial.print(memoryUsage);
    Serial.println(F(" B"));
    Serial.println();

    Serial.print(F("Encryption Voltage: "));
    Serial.print(encryptionVoltage);
    Serial.println(F(" V"));
    Serial.print(F("Encryption Current: "));
    Serial.print(encryptionCurrent);
    Serial.println(F(" mA"));
    Serial.print(F("Encryption Power: "));
    Serial.print(encryptionPower);
    Serial.println(F(" W"));
    Serial.println();
    
    Serial.print(F("Decryption Voltage: "));
    Serial.print(decryptionVoltage);
    Serial.println(F(" V"));
    Serial.print(F("Decryption Current: "));
    Serial.print(decryptionCurrent);
    Serial.println(F(" mA"));
    Serial.print(F("Decryption Power: "));
    Serial.print(decryptionPower);
    Serial.println(F(" W"));

    Serial.println(F("selesai"));
}}

void setup() {{
    Serial.begin(9600);
    SPI.begin();
    rfid.PCD_Init();
    ina219.begin();
    
    lcd.begin(16, 2);
    lcd.backlight();
    lcd.setCursor(0, 0);
    lcd.print("RFID System Ready");
    delay(2000);
    lcd.clear();
}}

void loop() {{
    if (rfid.PICC_IsNewCardPresent() && rfid.PICC_ReadCardSerial()) {{
        byte cardID[{id_values}] = {{0}};
        size_t idLength = rfid.uid.size;

        for (byte i = 0; i < idLength && i < {id_values}; i++) {{
            cardID[i] = rfid.uid.uidByte[i];
        }}

        if (idLength < {id_values}) {{
            memset(&cardID[idLength], 0, {id_values} - idLength);
        }}

        lcd.clear();
        lcd.setCursor(0, 0);
        lcd.print("RFID Detected!");
        lcd.setCursor(0, 1);
        lcd.print("Encrypting...");

        encryptAndPrint(&speck128, keySpeck128, sizeof(keySpeck128), cardID, {id_values});
        decryptAndPrint(&speck128, keySpeck128, sizeof(keySpeck128), ciphertext, {id_values});
        displayData();

        lcd.clear();
        lcd.setCursor(0, 0);
        lcd.print("Encryption Done!");
        delay(2000);
        lcd.clear();
    }}
}}
"""

    with open(file_path, 'w') as file:
        file.write(code_template)

    print(f"Code with {id_values}-byte plaintext has been written to {file_path}")

# Generate folders and codes
for folder in code_folders:
    for i in range(400, 15, -16):
        filename = f"speck128rfid_{i}"
        sketch_folder = os.path.join(folder, filename)
        os.makedirs(sketch_folder, exist_ok=True)  # Membuat folder untuk setiap file

        file_path = os.path.join(sketch_folder, f"{filename}.ino")

        generate_code_with_decrementing_plaintext(file_path, i)


Code with 400-byte plaintext has been written to speck128rfida\speck128rfid_400\speck128rfid_400.ino
Code with 384-byte plaintext has been written to speck128rfida\speck128rfid_384\speck128rfid_384.ino
Code with 368-byte plaintext has been written to speck128rfida\speck128rfid_368\speck128rfid_368.ino
Code with 352-byte plaintext has been written to speck128rfida\speck128rfid_352\speck128rfid_352.ino
Code with 336-byte plaintext has been written to speck128rfida\speck128rfid_336\speck128rfid_336.ino
Code with 320-byte plaintext has been written to speck128rfida\speck128rfid_320\speck128rfid_320.ino
Code with 304-byte plaintext has been written to speck128rfida\speck128rfid_304\speck128rfid_304.ino
Code with 288-byte plaintext has been written to speck128rfida\speck128rfid_288\speck128rfid_288.ino
Code with 272-byte plaintext has been written to speck128rfida\speck128rfid_272\speck128rfid_272.ino
Code with 256-byte plaintext has been written to speck128rfida\speck128rfid_256\speck128rfi

In [3]:
import subprocess
import serial
import csv
import re
import time
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# a

import os
import re
import subprocess
import time
import serial

# Configurations
port = "COM18"
baud_rate = 9600
base_folder = "speck128rfida"  # Hanya folder yang diinginkan
arduino_cli_path = r"C:\Program Files\Arduino IDE\resources\app\lib\backend\resources\arduino-cli.exe"
log_base_folder = "../../logs"

# Ensure the base log folder exists
os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    # Compile and upload code to Arduino
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    # Run compile command and capture output for memory usage
    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2)  # Wait for Arduino to reset

    # Extract flash and RAM usage from the compile output
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    # Write Flash and RAM info to the log file
    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    # Return success or failure based on the result of compile and upload
    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    # Capture serial output to a log file
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:  # Open in append mode
        ser.flushInput()  # Clear any leftover data in the buffer
        start_time = time.time()
        while time.time() - start_time < 10:  # 10-second timeout
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)  # Additional wait to ensure all data is captured

# Define log folder for the current base folder
log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)  # Create folder if it doesn't exist

# Get sorted list of subfolders in the base folder
subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        # Define log file path for the current sketch
        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        # Compile and upload; skip if failed
        if compile_and_upload(sketch_path, log_file):
            # Capture serial output
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


Running speck128rfid_16 in speck128rfida...
speck128rfid_16.ino - Flash: 15104 / 30720 bytes, RAM: 1107 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83
Encryption Completed in 460 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 472 us
Memory Usage = 1142 B

Encryption Voltage: 4.17 V
Encryption Current: 7.20 mA
Encryption Power: 0.03 W

Decryption Voltage: 4.16 V
Decryption Current: 7.20 mA
Decryption Power: 0.03 W
selesai
Running speck128rfid_32 in speck128rfida...
speck128rfid_32.ino - Flash: 15178 / 30720 bytes, RAM: 1139 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83 70 22 1A 53 4E 04 7E 53 3F 3A 24 89 94 A3 95 39
Encryption Completed in 636 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 656 us
Memory Usage = 1174 B

Encryption Voltage: 4.16 V
Encryption Current: 7.20 mA
Encryption Power: 0.03 W

De

In [ ]:
# b

import os
import re
import subprocess
import time
import serial

# Configurations
port = "COM18"
baud_rate = 9600
base_folder = "speck128rfidb"  # Hanya folder yang diinginkan
arduino_cli_path = r"C:\Program Files\Arduino IDE\resources\app\lib\backend\resources\arduino-cli.exe"
log_base_folder = "../../logs"

# Ensure the base log folder exists
os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    # Compile and upload code to Arduino
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    # Run compile command and capture output for memory usage
    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2)  # Wait for Arduino to reset

    # Extract flash and RAM usage from the compile output
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    # Write Flash and RAM info to the log file
    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    # Return success or failure based on the result of compile and upload
    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    # Capture serial output to a log file
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:  # Open in append mode
        ser.flushInput()  # Clear any leftover data in the buffer
        start_time = time.time()
        while time.time() - start_time < 10:  # 10-second timeout
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)  # Additional wait to ensure all data is captured

# Define log folder for the current base folder
log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)  # Create folder if it doesn't exist

# Get sorted list of subfolders in the base folder
subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        # Define log file path for the current sketch
        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        # Compile and upload; skip if failed
        if compile_and_upload(sketch_path, log_file):
            # Capture serial output
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


Running speck128rfid_16 in speck128rfidb...
speck128rfid_16.ino - Flash: 15104 / 30720 bytes, RAM: 1107 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83
Encryption Completed in 460 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 472 us
Memory Usage = 1142 B

Encryption Voltage: 4.16 V
Encryption Current: 7.00 mA
Encryption Power: 0.03 W

Decryption Voltage: 4.16 V
Decryption Current: 6.90 mA
Decryption Power: 0.03 W
selesai
Running speck128rfid_32 in speck128rfidb...
speck128rfid_32.ino - Flash: 15178 / 30720 bytes, RAM: 1139 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83 70 22 1A 53 4E 04 7E 53 3F 3A 24 89 94 A3 95 39
Encryption Completed in 636 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 656 us
Memory Usage = 1174 B

Encryption Voltage: 4.16 V
Encryption Current: 7.40 mA
Encryption Power: 0.03 W

De

In [7]:
# c

import os
import re
import subprocess
import time
import serial

# Configurations
port = "COM18"
baud_rate = 9600
base_folder = "speck128rfidc"  # Hanya folder yang diinginkan
arduino_cli_path = r"C:\Program Files\Arduino IDE\resources\app\lib\backend\resources\arduino-cli.exe"
log_base_folder = "../../logs"

# Ensure the base log folder exists
os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    # Compile and upload code to Arduino
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    # Run compile command and capture output for memory usage
    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2)  # Wait for Arduino to reset

    # Extract flash and RAM usage from the compile output
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    # Write Flash and RAM info to the log file
    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    # Return success or failure based on the result of compile and upload
    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    # Capture serial output to a log file
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:  # Open in append mode
        ser.flushInput()  # Clear any leftover data in the buffer
        start_time = time.time()
        while time.time() - start_time < 10:  # 10-second timeout
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)  # Additional wait to ensure all data is captured

# Define log folder for the current base folder
log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)  # Create folder if it doesn't exist

# Get sorted list of subfolders in the base folder
subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        # Define log file path for the current sketch
        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        # Compile and upload; skip if failed
        if compile_and_upload(sketch_path, log_file):
            # Capture serial output
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


Running speck128rfid_16 in speck128rfidc...
speck128rfid_16.ino - Flash: 15104 / 30720 bytes, RAM: 1107 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83
Encryption Completed in 460 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 472 us
Memory Usage = 1142 B

Encryption Voltage: 4.16 V
Encryption Current: 7.10 mA
Encryption Power: 0.03 W

Decryption Voltage: 4.16 V
Decryption Current: 7.00 mA
Decryption Power: 0.03 W
selesai
Running speck128rfid_32 in speck128rfidc...
speck128rfid_32.ino - Flash: 15178 / 30720 bytes, RAM: 1139 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83 70 22 1A 53 4E 04 7E 53 3F 3A 24 89 94 A3 95 39
Encryption Completed in 636 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 656 us
Memory Usage = 1174 B

Encryption Voltage: 4.17 V
Encryption Current: 7.10 mA
Encryption Power: 0.03 W

De

In [8]:
# d

import os
import re
import subprocess
import time
import serial

# Configurations
port = "COM18"
baud_rate = 9600
base_folder = "speck128rfidd"  # Hanya folder yang diinginkan
arduino_cli_path = r"C:\Program Files\Arduino IDE\resources\app\lib\backend\resources\arduino-cli.exe"
log_base_folder = "../../logs"

# Ensure the base log folder exists
os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    # Compile and upload code to Arduino
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    # Run compile command and capture output for memory usage
    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2)  # Wait for Arduino to reset

    # Extract flash and RAM usage from the compile output
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    # Write Flash and RAM info to the log file
    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    # Return success or failure based on the result of compile and upload
    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    # Capture serial output to a log file
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:  # Open in append mode
        ser.flushInput()  # Clear any leftover data in the buffer
        start_time = time.time()
        while time.time() - start_time < 10:  # 10-second timeout
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)  # Additional wait to ensure all data is captured

# Define log folder for the current base folder
log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)  # Create folder if it doesn't exist

# Get sorted list of subfolders in the base folder
subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        # Define log file path for the current sketch
        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        # Compile and upload; skip if failed
        if compile_and_upload(sketch_path, log_file):
            # Capture serial output
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


Running speck128rfid_16 in speck128rfidd...
speck128rfid_16.ino - Flash: 15104 / 30720 bytes, RAM: 1107 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83
Encryption Completed in 460 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 472 us
Memory Usage = 1142 B

Encryption Voltage: 4.16 V
Encryption Current: 7.00 mA
Encryption Power: 0.03 W

Decryption Voltage: 4.16 V
Decryption Current: 7.10 mA
Decryption Power: 0.03 W
selesai
Running speck128rfid_32 in speck128rfidd...
speck128rfid_32.ino - Flash: 15178 / 30720 bytes, RAM: 1139 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83 70 22 1A 53 4E 04 7E 53 3F 3A 24 89 94 A3 95 39
Encryption Completed in 636 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 656 us
Memory Usage = 1174 B

Encryption Voltage: 4.16 V
Encryption Current: 6.80 mA
Encryption Power: 0.03 W

De

In [10]:
# e

import os
import re
import subprocess
import time
import serial

# Configurations
port = "COM18"
baud_rate = 9600
base_folder = "speck128rfide"  # Hanya folder yang diinginkan
arduino_cli_path = r"C:\Program Files\Arduino IDE\resources\app\lib\backend\resources\arduino-cli.exe"
log_base_folder = "../../logs"

# Ensure the base log folder exists
os.makedirs(log_base_folder, exist_ok=True)

def compile_and_upload(sketch_path, log_file):
    # Compile and upload code to Arduino
    compile_command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:nano", sketch_path]
    upload_command = [arduino_cli_path, "upload", "-p", port, "--fqbn", "arduino:avr:nano", sketch_path]

    # Run compile command and capture output for memory usage
    compile_result = subprocess.run(compile_command, capture_output=True, text=True)
    upload_result = subprocess.run(upload_command, capture_output=True, text=True)
    time.sleep(2)  # Wait for Arduino to reset

    # Extract flash and RAM usage from the compile output
    flash_usage = ram_usage = None
    flash_match = re.search(r"Sketch uses (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    ram_match = re.search(r"Global variables use (\d+) bytes.*Maximum is (\d+) bytes", compile_result.stdout)
    if flash_match and ram_match:
        flash_usage = f"{flash_match.group(1)} / {flash_match.group(2)} bytes"
        ram_usage = f"{ram_match.group(1)} / {ram_match.group(2)} bytes"
        print(f"{os.path.basename(sketch_path)} - Flash: {flash_usage}, RAM: {ram_usage}")
    else:
        print(f"{os.path.basename(sketch_path)} - Failed to retrieve memory usage information")

    # Write Flash and RAM info to the log file
    with open(log_file, "w") as f:
        if flash_usage and ram_usage:
            f.write(f"Flash Usage: {flash_usage}, RAM Usage: {ram_usage}\n\n")
        else:
            f.write("Failed to retrieve memory usage information\n\n")

    # Return success or failure based on the result of compile and upload
    return compile_result.returncode == 0 and upload_result.returncode == 0

def record_serial_data(log_file):
    # Capture serial output to a log file
    with serial.Serial(port, baud_rate, timeout=1) as ser, open(log_file, 'a') as file:  # Open in append mode
        ser.flushInput()  # Clear any leftover data in the buffer
        start_time = time.time()
        while time.time() - start_time < 10:  # 10-second timeout
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').strip()
                file.write(line + '\n')
                print(line)
                if 'selesai' in line.lower():
                    break
        time.sleep(1)  # Additional wait to ensure all data is captured

# Define log folder for the current base folder
log_folder = os.path.join(log_base_folder, f"logs_{base_folder}")
os.makedirs(log_folder, exist_ok=True)  # Create folder if it doesn't exist

# Get sorted list of subfolders in the base folder
subfolders = sorted(
    [folder for folder in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, folder))],
    key=lambda x: int(x.split('_')[1])
)

for folder_name in subfolders:
    sketch_path = os.path.join(base_folder, folder_name, f"{folder_name}.ino")
    if os.path.isfile(sketch_path):
        print(f"Running {folder_name} in {base_folder}...")

        # Define log file path for the current sketch
        log_file = os.path.join(log_folder, f"{folder_name}.txt")

        # Compile and upload; skip if failed
        if compile_and_upload(sketch_path, log_file):
            # Capture serial output
            record_serial_data(log_file)
        else:
            print(f"{folder_name} failed to run in {base_folder}.\n")
    else:
        print(f"File {folder_name}.ino not found in {folder_name} within {base_folder}")

print("Data collection complete.")


Running speck128rfid_16 in speck128rfide...
speck128rfid_16.ino - Flash: 15104 / 30720 bytes, RAM: 1107 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83
Encryption Completed in 460 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 472 us
Memory Usage = 1142 B

Encryption Voltage: 4.16 V
Encryption Current: 7.00 mA
Encryption Power: 0.03 W

Decryption Voltage: 4.16 V
Decryption Current: 7.10 mA
Decryption Power: 0.03 W
selesai
Running speck128rfid_32 in speck128rfide...
speck128rfid_32.ino - Flash: 15178 / 30720 bytes, RAM: 1139 / 2048 bytes
Ciphertext: 89 51 BC 94 2A A2 70 D5 3F C7 D2 88 59 E8 A9 83 70 22 1A 53 4E 04 7E 53 3F 3A 24 89 94 A3 95 39
Encryption Completed in 636 us
Decrypted Text: 23 EE AE 0D 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
Decryption Completed in 656 us
Memory Usage = 1174 B

Encryption Voltage: 4.16 V
Encryption Current: 7.10 mA
Encryption Power: 0.03 W

De

In [5]:
import os
import re
import csv

# Define the main directory paths
log_base_folder = "../../logs"
output_base_folder = "../../Data_CSV_128/CSVs_128"

# Ensure the output folder exists
os.makedirs(output_base_folder, exist_ok=True)

# Define the header for the CSV files
header = [
    "file_name", "flash_usage", "ram_usage", "memory_usage",
    "encryption_time", "encryption_voltage", "encryption_current", "encryption_power",
    "decryption_time", "decryption_voltage", "decryption_current", "decryption_power"
]

# Function to parse individual .txt files
def parse_txt_file(file_path):
    with open(file_path, 'r') as f:
        data = f.read()

    # Extract flash usage and RAM usage, capturing only the number before "/"
    flash_match = re.search(r"Flash Usage: (\d+) / \d+ bytes", data)
    ram_match = re.search(r"RAM Usage: (\d+) / \d+ bytes", data)
    memory_match = re.search(r"Memory Usage = (\d+) B", data)

    # Extract encryption and decryption times
    encryption_time_match = re.search(r"Encryption Completed in (\d+) us", data)
    decryption_time_match = re.search(r"Decryption Completed in (\d+) us", data)

    # Extract encryption and decryption voltages and currents
    encryption_voltage_match = re.search(r"Encryption Voltage: ([\d.]+) V", data)
    encryption_current_match = re.search(r"Encryption Current: ([\d.]+) mA", data)
    decryption_voltage_match = re.search(r"Decryption Voltage: ([\d.]+) V", data)
    decryption_current_match = re.search(r"Decryption Current: ([\d.]+) mA", data)

    # Extract values if matches are found, else set as None
    flash_usage = int(flash_match.group(1)) if flash_match else None
    ram_usage = int(ram_match.group(1)) if ram_match else None
    memory_usage = int(memory_match.group(1)) if memory_match else None
    encryption_time = int(encryption_time_match.group(1)) if encryption_time_match else None
    decryption_time = int(decryption_time_match.group(1)) if decryption_time_match else None
    encryption_voltage = float(encryption_voltage_match.group(1)) if encryption_voltage_match else None
    encryption_current = float(encryption_current_match.group(1)) if encryption_current_match else None
    decryption_voltage = float(decryption_voltage_match.group(1)) if decryption_voltage_match else None
    decryption_current = float(decryption_current_match.group(1)) if decryption_current_match else None

    # Calculate encryption and decryption power
    encryption_power = encryption_voltage * (encryption_current / 1000) if encryption_voltage and encryption_current else None
    decryption_power = decryption_voltage * (decryption_current / 1000) if decryption_voltage and decryption_current else None

    # Return all extracted data as a dictionary
    return {
        "file_name": os.path.splitext(os.path.basename(file_path))[0],  # Remove ".txt" from file name
        "flash_usage": flash_usage,
        "ram_usage": ram_usage,
        "memory_usage": memory_usage,
        "encryption_time": encryption_time,
        "encryption_voltage": encryption_voltage,
        "encryption_current": encryption_current,
        "encryption_power": encryption_power,
        "decryption_time": decryption_time,
        "decryption_voltage": decryption_voltage,
        "decryption_current": decryption_current,
        "decryption_power": decryption_power
    }

# Loop through each log folder
for log_folder in os.listdir(log_base_folder):
    log_folder_path = os.path.join(log_base_folder, log_folder)
    if os.path.isdir(log_folder_path) and log_folder.startswith("logs_speck128rfid"):
        # Define output CSV file path for each folder
        output_csv = os.path.join(output_base_folder, f"{log_folder}.csv")

        # List to hold all parsed data for this folder
        all_data = []

        # Retrieve and sort .txt files numerically based on the number after "aes128rfid_"
        txt_files = sorted(
            [file for file in os.listdir(log_folder_path) if file.endswith(".txt")],
            key=lambda x: int(re.search(r'speck128rfid_(\d+)', x).group(1))  # Extract the number after "aes128rfid_" and sort numerically
        )

        # Process each sorted .txt file
        for file_name in txt_files:
            file_path = os.path.join(log_folder_path, file_name)
            file_data = parse_txt_file(file_path)
            all_data.append(file_data)

        # Write all sorted data to CSV
        with open(output_csv, 'w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=header)
            writer.writeheader()
            writer.writerows(all_data)

        print(f"Data from {log_folder} has been successfully saved to {output_csv} in the correct order.")


Data from logs_speck128rfida has been successfully saved to ../../Data_CSV_128/CSVs_128\logs_speck128rfida.csv in the correct order.
Data from logs_speck128rfidb has been successfully saved to ../../Data_CSV_128/CSVs_128\logs_speck128rfidb.csv in the correct order.
Data from logs_speck128rfidc has been successfully saved to ../../Data_CSV_128/CSVs_128\logs_speck128rfidc.csv in the correct order.
Data from logs_speck128rfidd has been successfully saved to ../../Data_CSV_128/CSVs_128\logs_speck128rfidd.csv in the correct order.
Data from logs_speck128rfide has been successfully saved to ../../Data_CSV_128/CSVs_128\logs_speck128rfide.csv in the correct order.


In [11]:
import os
import re
import csv

# Define the main directory paths
log_base_folder = "../../logs"
output_base_folder = "../../Data_CSV_128/CSVs_128_chippertext"

# Ensure the output folder exists
os.makedirs(output_base_folder, exist_ok=True)

# Define the header for the CSV files
header = [
    "file_name", "ciphertext", "decrypted_text"
]

# Function to parse individual .txt files
def parse_txt_file(file_path):
    with open(file_path, 'r') as f:
        data = f.read()

    # Extract ciphertext and decrypted text
    ciphertext_match = re.search(r"Ciphertext: ([0-9A-Fa-f ]+)", data)
    decrypted_text_match = re.search(r"Decrypted Text: ([0-9A-Fa-f ]+)", data)

    # Extract values if matches are found, else set as None
    ciphertext = ciphertext_match.group(1).strip() if ciphertext_match else None
    decrypted_text = decrypted_text_match.group(1).strip() if decrypted_text_match else None

    # Return all extracted data as a dictionary
    return {
        "file_name": os.path.splitext(os.path.basename(file_path))[0],  # Remove ".txt" from file name
        "ciphertext": ciphertext,
        "decrypted_text": decrypted_text
    }

# Loop through each log folder
for log_folder in os.listdir(log_base_folder):
    log_folder_path = os.path.join(log_base_folder, log_folder)
    if os.path.isdir(log_folder_path) and log_folder.startswith("logs_speck128rfid"):
        # Define output CSV file path for each folder
        output_csv = os.path.join(output_base_folder, f"{log_folder}.csv")

        # List to hold all parsed data for this folder
        all_data = []

        # Retrieve and sort .txt files numerically based on the number after "speck128rfid_"
        txt_files = sorted(
            [file for file in os.listdir(log_folder_path) if file.endswith(".txt")],
            key=lambda x: int(re.search(r'speck128rfid_(\d+)', x).group(1))  # Extract the number after "speck128rfid_" and sort numerically
        )

        # Process each sorted .txt file
        for file_name in txt_files:
            file_path = os.path.join(log_folder_path, file_name)
            file_data = parse_txt_file(file_path)
            all_data.append(file_data)

        # Write all sorted data to CSV
        with open(output_csv, 'w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=header)
            writer.writeheader()
            writer.writerows(all_data)

        print(f"Data from {log_folder} has been successfully saved to {output_csv} in the correct order.")


Data from logs_speck128rfida has been successfully saved to ../../Data_CSV_128/CSVs_128_chippertext\logs_speck128rfida.csv in the correct order.
Data from logs_speck128rfidb has been successfully saved to ../../Data_CSV_128/CSVs_128_chippertext\logs_speck128rfidb.csv in the correct order.
Data from logs_speck128rfidc has been successfully saved to ../../Data_CSV_128/CSVs_128_chippertext\logs_speck128rfidc.csv in the correct order.
Data from logs_speck128rfidd has been successfully saved to ../../Data_CSV_128/CSVs_128_chippertext\logs_speck128rfidd.csv in the correct order.
Data from logs_speck128rfide has been successfully saved to ../../Data_CSV_128/CSVs_128_chippertext\logs_speck128rfide.csv in the correct order.


In [6]:
import pandas as pd
import glob
import os
import re

# Tentukan direktori yang berisi file CSV
directory = "../../Data_CSV_128/CSVs_128"

# Tentukan output path untuk menyimpan hasil rata-rata
output_path = "../../Data_CSV_128/avg_csv_128/avg_csv_128.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Temukan semua file CSV di dalam folder
file_paths = glob.glob(os.path.join(directory, '*.csv'))

# Periksa apakah ada file CSV yang ditemukan
if not file_paths:
    print("No CSV files found in the specified directory.")
else:
    # Baca semua file CSV dan simpan dalam daftar DataFrame
    dfs = [pd.read_csv(file) for file in file_paths]
    
    # Gunakan file pertama sebagai dasar untuk struktur output
    average_df = dfs[0].copy()

    # Hitung rata-rata untuk setiap kolom numerik di semua file
    for col in average_df.select_dtypes(include='number').columns:
        average_df[col] = sum(df[col] for df in dfs) / len(dfs)

    # Sort file_name secara numerik berdasarkan angka di dalam nama
    average_df['sort_key'] = average_df['file_name'].str.extract(r'(\d+)').astype(int)
    sorted_df = average_df.sort_values('sort_key').drop(columns=['sort_key']).reset_index(drop=True)
    
    # Simpan hasil rata-rata ke file CSV baru
    sorted_df.to_csv(output_path, index=False)
    print(f"Averaged and sorted CSV file saved to: {output_path}")


Averaged and sorted CSV file saved to: ../../Data_CSV_128/avg_csv_128/avg_csv_128.csv
